<a href="https://colab.research.google.com/github/dasmiq/cs6120-assignment2/blob/main/reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Sentiment Labels

In this experiment, you will explore the accuracy of sentiment classificaiton using different feature representations of text documents.

First, you will implement `createBasicFeatures`, which creates a sparse matrix representation of a collection of documents.  As we discussed in class, many classification models represent a document as a sparse vector of of features and their weights. Weights could be boolean, counts, tf-idf values, or other functions of the document. In a linear model, we then combine the document vectors with the candidate output classes and learn weights for each of the feature-class pairs. This last step is implemented for you.

For this exercise, you should have a feature for each word containing at least one alphabetic character. You may use the `numpy` and `sklearn` packages to help with implementing a sparse matrix.

Then, you will implement `createFancyFeatures`, which can specify at any other features you choose to help improve performance on the classification task.

The two code blocks at the end train and evaluate two models—logistic regression with L1 and L2 regularization—using your featurization functions. Besides held-out classification accuracy with 10-fold cross-validation, you will also see the features in each class given high weights by the model.

There are many helpful resources online for getting up to speed with vector representations of documents. One example is the first two chapters of Delip Rao and Brian McMahan, _Natural Language Processing with PyTorch_, O'Reilly, 2019.  You should be able to <a href="https://learning.oreilly.com/library/view/natural-language-processing/9781491978221/">read it online</a> via the Northeastern Library's subscription using a <tt>northeastern.edu</tt> email address.

In [15]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
import re
from scipy.sparse import lil_matrix

In [16]:
# read in the movie review corpus
def readReviews():
  raw = requests.get("https://raw.githubusercontent.com/dasmiq/cs6120-assignment2/refs/heads/main/cornell_reviews.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  return corpus

This is where you will implement two functions to featurize the data.

In [17]:
# TODO: Implement createBasicFeatures
# NB: The current contents are for testing only
# This function should return:
#  -a sparse numpy matrix of document features
#  -a list of the correct class for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix.

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
def createBasicFeatures(corpus):
  # collect the tokens
  all_tokens = set()
  tokenized_docs = []

  for doc in corpus:
    text = doc["text"].lower()
    tokens = re.findall(r"[a-zA-Z]+", text)
    tokenized_docs.append(tokens)
    all_tokens.update(tokens)

  # build vocabulary 
  vocab = sorted(all_tokens)
  token_index = {token: idx for idx, token in enumerate(vocab)}

  # create sparse matrix
  num_docs = len(corpus)
  num_vocab = len(vocab)
  texts = lil_matrix((num_docs, num_vocab), dtype=int)
  for doc_id, tokens in enumerate(tokenized_docs):
      for token in tokens:
          if token in token_index:
              texts[doc_id, token_index[token]] += 1

  # collect classes
  classes = [doc["class"] for doc in corpus]

  return texts,classes,vocab

In [ ]:
# TODO: Implement createFancyFeatures andn describe in comments what the
# features are and why they might ben helpful.
# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
def createFancyFeatures(corpus):
    
    """
    Create feature matrix with additional features beyond unigrams.
    Features included:
    1. Unigrams (basic word counts): core representation of text.
    2. Bigrams (two-word sequences): capture local context, e.g. "not good".
    3. Exclamation count: captures emotional intensity, often correlated with sentiment.
    """
    
    # collect the tokens
    tokenized_docs = []
    all_features = set()
    extra_features = ["<EXCLAMATION_COUNT>"]

    for doc in corpus:
        text = doc["text"].lower()
        tokens = re.findall(r"[a-zA-Z]+", text)
        features = []

        # add unigrams
        features.extend(tokens)

        for i in range(len(tokens) - 1):
            bigram = tokens[i] + "_" + tokens[i+1]
            features.append(bigram)

        # add exclamation mark count as a pseudo-token
        exclam_count = doc["text"].count("!")
        if exclam_count > 0:
            features.append("<EXCLAMATION_COUNT>")

        tokenized_docs.append(features)
        all_features.update(features)

    # build vocabulary (unigrams + bigrams + special features)
    vocab = sorted(list(all_features))
    feature2idx = {f: i for i, f in enumerate(vocab)}

    # create sparse matrix
    num_docs = len(corpus)
    num_vocab = len(vocab)
    texts = lil_matrix((num_docs, num_vocab), dtype=int)

    for doc_id, features in enumerate(tokenized_docs):
        for f in features:
            if f in feature2idx:
                texts[doc_id, feature2idx[f]] += 1
    
    # collect labels
    classes = [doc["class"] for doc in corpus]
  
    return texts,classes,vocab

In [19]:
# given a numpy matrix representation of the features for the training set, the
# vector of true classes for each example, and the vocabulary as described
# above, this computes the accuracy of the model using leave one out cross
# validation and reports the most indicative features for each class

def evaluateModel(X,y,vocab,penalty="l1"):
  # create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=KFold(n_splits=10, shuffle=True, random_state=1))

  # determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)

  # determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  class0_weight_sorted = model.coef_[0, :].argsort()
  class1_weight_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  class0_indicators = [vocab[i] for i in class0_weight_sorted[:termsToTake]]
  class1_indicators = [vocab[i] for i in class1_weight_sorted[:termsToTake]]

  if model.classes_[0] == "pos":
    return avg_score,class0_indicators,class1_indicators
  else:
    return avg_score,class1_indicators,class0_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [20]:
corpus = readReviews()

Run the following to train and evaluate two models using basic features:

In [21]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.826500
The most informative terms for pos are: ['flaws', 'memorable', 'terrific', 'perfectly', 'edge', 'enjoyable', 'sherri', 'masterpiece', 'excellent', 'gas', 'overall', 'using', 'quite', 'fun', 'follows', 'pulp', 'solid', 'holds', 'seen', 'loved']
The most informative terms for neg are: ['waste', 'mess', 'ridiculous', 'lame', 'worst', 'headed', 'awful', 'unfortunately', 'cheap', 'write', 'boring', 'superior', 'tedious', 'bad', 'jesse', 'terrible', 'poor', 'looks', 'maybe', 'reason']
----------L2 Norm-----------
The model's average accuracy is 0.832500
The most informative terms for pos are: ['fun', 'back', 'great', 'quite', 'well', 'excellent', 'overall', 'seen', 'american', 'perfectly', 'memorable', 'terrific', 'yet', 'pulp', 'job', 'true', 'performances', 'very', 'different', 'solid']
The most informative terms for neg are: ['bad', 'unfortunately', 'worst', 'waste', 'nothing', 'only', 'script', 'boring', 'awful', 'plot

Run the following to train and evaluate two models using extended features:

In [22]:
X,y,vocab = createFancyFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.829500
The most informative terms for pos are: ['even_if', 'flaws', 'memorable', 'terrific', 'due_to', 'overall', 'the_more', 'seen', 'masterpiece', 'follows', 'by_a', 'gas', 'loved', 'others', 'her_husband', 'as_much', 'works', 'just_about', 'filmmaking', 'great']
The most informative terms for neg are: ['ridiculous', 'waste', 'unfortunately', 'worst', 'awful', 'mess', 'cheap', 'boring', 'lame', 'should_have', 'headed', 'poor', 'bad', 'write', 'maybe', 'designed', 'superior', 'nowhere', 'terrible', 'metro']
----------L2 Norm-----------
The model's average accuracy is 0.851500
The most informative terms for pos are: ['great', 'well', 'fun', 'seen', 'very', 'back', 'also', 'quite', 'people', 'many', 'yet', 'american', 'while', 'life', 'see', 'the_two', 'job', 'excellent', 'most', 'perfectly']
The most informative terms for neg are: ['bad', 'only', 'unfortunately', 'worst', 'plot', 'nothing', 'any', 'boring', 'waste', 'script

**TODO**: Briefly comment on your results. You do _not_ need to beat the basic features model to get full credit.

1. The basic model has the similar accuracy of L1 Norm to the result of the fancy model, both are around 83%. But in the fancy model, the accuracy of L2 regularization is 85%, higher than the basic model in L2 Norm which is 83.25%.
2. The most positive terms include words like "great", "fun", and "excellent", while the most negative terms include "bad", "worst", and "waste". In the fancy feature model, bigram features such as "to_be" and "but_the" also appear, showing that contextual features are being captured. 
3. Overall, the fancy model provides a small improvement, especially in L2 Norm, but even the basic unigram features are already quite effective for sentiment classification.